## Fake News Classifier Using LSTM


In [20]:
import pandas as pd
import csv

In [23]:
df = pd.read_csv('train.csv', on_bad_lines='skip', encoding='utf-8')

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [24]:
df.shape

(20800, 5)

In [25]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [26]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [27]:
### Drop Nan Values
df=df.dropna()


In [28]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [29]:
## Get The Independent Features
X=df.drop('label',axis=1)

In [30]:
## Get The Dependent Features
y=df['label']

In [31]:
X.shape

(18285, 4)

In [32]:
y.shape

(18285,)

In [33]:
import tensorflow as tf

In [34]:
tf.__version__

'2.15.0'

In [35]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [36]:
### Vocabulary Size
voc_size=5000

### One Hot Representation

In [37]:
messages=X.copy()

In [38]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [39]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [41]:
messages.reset_index(inplace=True)

In [42]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [43]:
import nltk
import re
from nltk.corpus import stopwords

In [44]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [45]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [46]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [47]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [48]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[2199, 110, 4429, 2907, 3487, 1662, 941, 3372, 1000, 1315],
 [4914, 4909, 2650, 2131, 1638, 4506, 4625],
 [1209, 1007, 4060, 2564],
 [2196, 279, 1208, 3345, 864, 2431],
 [1942, 1638, 4854, 2314, 3601, 902, 1638, 294, 825, 1970],
 [1137,
  3062,
  474,
  3864,
  621,
  1659,
  4468,
  865,
  1572,
  4289,
  4351,
  1715,
  3472,
  665,
  4625],
 [4129, 2208, 996, 755, 4122, 393, 3071, 4718, 2675, 2646, 662],
 [1576, 3289, 3886, 1768, 1768, 3598, 1659, 1957, 2675, 2646, 662],
 [2838, 4389, 257, 959, 3651, 63, 3266, 4226, 1659, 1164],
 [684, 1294, 1066, 1474, 1043, 4253, 3274, 4478],
 [4788, 2678, 2233, 4168, 2963, 2144, 3800, 3764, 1104, 1041, 271],
 [3345, 243, 3487, 63, 1659, 1768],
 [1367, 1235, 1960, 4395, 2428, 2435, 3309, 3277, 3500],
 [342, 4155, 777, 4797, 1369, 1992, 4416, 2675, 2646, 662],
 [2184, 3684, 58, 4857, 2114, 2675, 2646, 662],
 [3001, 4907, 2981, 1491, 1609, 17, 4355, 2684, 4086, 273],
 [3002, 2450, 4909],
 [2983, 4551, 3297, 4946, 1659, 55, 1653, 4625],
 [529, 4161,

In [49]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [50]:
onehot_repr[1]

[4914, 4909, 2650, 2131, 1638, 4506, 4625]

### Embedding Representation

In [51]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[2199  110 4429 ...    0    0    0]
 [4914 4909 2650 ...    0    0    0]
 [1209 1007 4060 ...    0    0    0]
 ...
 [ 326 3673 1596 ...    0    0    0]
 [2616 1768  262 ...    0    0    0]
 [ 395   17 4460 ...    0    0    0]]


In [52]:
embedded_docs[1]

array([4914, 4909, 2650, 2131, 1638, 4506, 4625,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [53]:
embedded_docs[0]

array([2199,  110, 4429, 2907, 3487, 1662,  941, 3372, 1000, 1315,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [54]:
## Creating Model
embedding_vector_features=40 ## Features Representation 
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [55]:
len(embedded_docs),y.shape

(18285, (18285,))

In [56]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [57]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [58]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=40)

### Model Training

In [59]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 15s 62ms/step - loss: 0.3380 - accuracy: 0.8311 - val_loss: 0.1986 - val_accuracy: 0.9173
Epoch 2/10
192/192 [==============================] - 4s 20ms/step - loss: 0.1318 - accuracy: 0.9488 - val_loss: 0.2104 - val_accuracy: 0.9137
Epoch 3/10
192/192 [==============================] - 3s 13ms/step - loss: 0.0801 - accuracy: 0.9721 - val_loss: 0.2231 - val_accuracy: 0.9167
Epoch 4/10
192/192 [==============================] - 2s 10ms/step - loss: 0.0567 - accuracy: 0.9808 - val_loss: 0.2718 - val_accuracy: 0.9112
Epoch 5/10
192/192 [==============================] - 1s 8ms/step - loss: 0.0379 - accuracy: 0.9872 - val_loss: 0.3220 - val_accuracy: 0.9109
Epoch 6/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0228 - accuracy: 0.9938 - val_loss: 0.3526 - val_accuracy: 0.9102
Epoch 7/10
192/192 [==============================] - 2s 9ms/step - loss: 0.0198 - accuracy: 0.9936 - val_loss: 0.3931 - val_accuracy: 0.9064
E

### Adding Dropout

In [60]:
from tensorflow.keras.layers import Dropout
## Creating Model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

### Performance Metrics And Accuracy

In [61]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 2ms/step


In [62]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [63]:
from sklearn.metrics import confusion_matrix

In [64]:
confusion_matrix(y_test,y_pred)

array([[3394,    0],
       [2641,    0]])

In [65]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5623860811930406

In [66]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.56      1.00      0.72      3394
           1       0.00      0.00      0.00      2641

    accuracy                           0.56      6035
   macro avg       0.28      0.50      0.36      6035
weighted avg       0.32      0.56      0.40      6035



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
